In [0]:
from pyspark.sql.functions import current_timestamp, col

In [0]:
%run ../config/path_variables

In [0]:
races_df = spark.read.format('delta').load(f"{processed_file_path}/races")



In [0]:
circuits_df = spark.read.format('delta').load(f"{processed_file_path}/circuits")


In [0]:
drivers_df = spark.read.format('delta').load(f"{processed_file_path}/drivers")


In [0]:
results_df = spark.read.format('delta').load(f"{processed_file_path}/results")
results_df.dropDuplicates(["race_id", "driver_id"]).show()


In [0]:
constructors_df = spark.read.format('delta').load(f"{processed_file_path}/constructors")

In [0]:
final_df = races_df.alias("r") \
    .join(circuits_df.alias("c"), col("r.circuit_id") == col("c.circuit_id"), "inner") \
    .join(results_df.alias("res"), col("res.race_id") == col("r.race_id"), "inner") \
    .join(drivers_df.alias("d"), col("d.driver_id") == col("res.driver_id"), "inner") \
    .join(constructors_df.alias("con"), col("con.constructor_id") == col("res.constructor_id"), "inner") \
    .select(
        col("r.year").alias("race_year"),
        col("r.name").alias("race_name"),
        col("r.date").alias("race_date"),
        col("c.location").alias("curciut_location"),
        col("d.name").alias("driver_name"),
        col("d.number").alias("driver_number"),
        col("d.nationality").alias("driver_nationality"),
        col("con.name").alias("team"),
        col("res.grid"),
        col("res.fastest_lap"),
        col("res.time").alias("race_time"),
        col("res.points"),
        col("res.position")
    ) \
    .withColumn("created_date", current_timestamp())
    


In [0]:
display(final_df.filter("race_year == 2020 AND race_name == 'Abu Dhabi Grand Prix'").orderBy(final_df.points.desc()))

In [0]:
final_df.write.mode("overwrite").format("delta").saveAsTable("f1_presentation.race_results")

